<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br></center></h1>

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [3]:
import scipy.io as sio
from libsvm.svmutil import *

data = sio.loadmat('data.mat')

trainX = data['X'][:150]
trainY = [d[0] for d in data['Y'][:150]]
testX = data['X'][150:]
testY = [d[0] for d in data['Y'][150:]]

prob = svm_problem(trainY, trainX)

param = svm_parameter('-t 0 -c 10e20')
m = svm_train(prob, param)

print('Training Set:')
p_labels, p_acc, p_vals = svm_predict(trainY, trainX, m)

print('Test Set:')
p_labels, p_acc, p_vals = svm_predict(testY, testX, m)

Training Set:
Accuracy = 74.6667% (112/150) (classification)
Test Set:
Accuracy = 77.5% (93/120) (classification)


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

In [11]:
from prettytable import PrettyTable

print('Linear')
t = PrettyTable(['Parameters', 'Train Accuracy', 'Test Accuracy'])

for cExp in range(-10,11):
    paramConfig = '-t 0 -c '+str(5**cExp)
    param = svm_parameter(paramConfig)
    m = svm_train(prob, param)
    p_labelsTrain, p_accTrain, p_valsTrain = svm_predict(trainY, trainX, m, '-q')
    p_labelsTest, p_accTest, p_valsTest = svm_predict(testY, testX, m, '-q')
    t.add_row([paramConfig, p_accTrain[0], p_accTest[0]])

print(t.get_string(sortby="Test Accuracy", reversesort=True))

Linear
+-------------------+--------------------+--------------------+
|     Parameters    |   Train Accuracy   |   Test Accuracy    |
+-------------------+--------------------+--------------------+
|     -t 0 -c 1     | 86.66666666666667  |        85.0        |
|   -t 0 -c 390625  | 88.66666666666667  | 84.16666666666667  |
|    -t 0 -c 0.2    | 85.33333333333334  | 84.16666666666667  |
|    -t 0 -c 0.04   |        84.0        | 84.16666666666667  |
|   -t 0 -c 0.008   | 82.66666666666667  | 84.16666666666667  |
|  -t 0 -c 1953125  |        80.0        | 83.33333333333334  |
|   -t 0 -c 78125   | 89.33333333333333  |        82.5        |
|    -t 0 -c 625    |        90.0        | 81.66666666666667  |
|     -t 0 -c 5     | 88.66666666666667  | 81.66666666666667  |
|    -t 0 -c 3125   |        90.0        | 81.66666666666667  |
|     -t 0 -c 25    | 88.66666666666667  | 81.66666666666667  |
|   -t 0 -c 15625   |        90.0        | 81.66666666666667  |
|    -t 0 -c 125    | 88.66666666

In [12]:
print('Polynomial')
t = PrettyTable(['Parameters', 'Train Accuracy', 'Test Accuracy'])

for cExp in range(-10,11):
    for gammaExp in range(-5,3):
        for coef0Exp in range(-5,3):
            for degree in range(2,5):
                paramConfig = '-t 1 -c '+str(5**cExp)+' -g '+str(5**gammaExp)+' -r '+str(5**coef0Exp)+' -d '+str(degree)
                param = svm_parameter(paramConfig)
                m = svm_train(prob, param)
                p_labelsTrain, p_accTrain, p_valsTrain = svm_predict(trainY, trainX, m, '-q')
                p_labelsTest, p_accTest, p_valsTest = svm_predict(testY, testX, m, '-q')
                t.add_row([paramConfig, p_accTrain[0], p_accTest[0]])
                
print(t.get_string(sortby="Test Accuracy", reversesort=True))

Polynomial
+----------------------------------------------+--------------------+--------------------+
|                  Parameters                  |   Train Accuracy   |   Test Accuracy    |
+----------------------------------------------+--------------------+--------------------+
|     -t 1 -c 78125 -g 0.00032 -r 0.2 -d 4     | 88.66666666666667  | 85.83333333333333  |
|      -t 1 -c 625 -g 0.00032 -r 0.2 -d 3      | 84.66666666666667  | 85.83333333333333  |
|      -t 1 -c 6.4e-05 -g 0.2 -r 25 -d 3       | 84.66666666666667  | 85.83333333333333  |
|        -t 1 -c 5 -g 0.0016 -r 1 -d 3         | 84.66666666666667  | 85.83333333333333  |
|       -t 1 -c 25 -g 0.00032 -r 1 -d 3        | 84.66666666666667  | 85.83333333333333  |
|     -t 1 -c 15625 -g 0.00032 -r 0.2 -d 3     |        88.0        | 85.83333333333333  |
|    -t 1 -c 15625 -g 0.00032 -r 0.04 -d 3     | 84.66666666666667  | 85.83333333333333  |
|       -t 1 -c 125 -g 0.0016 -r 1 -d 4        | 88.66666666666667  | 85.833333

In [13]:
print('Radial Basis Function')
t = PrettyTable(['Parameters', 'Train Accuracy', 'Test Accuracy'])

for cExp in range(-10,11):
    for gammaExp in range(-10,11):
        paramConfig = '-t 2 -c '+str(5**cExp)+' -g '+str(5**gammaExp)
        param = svm_parameter(paramConfig)
        m = svm_train(prob, param)
        p_labelsTrain, p_accTrain, p_valsTrain = svm_predict(trainY, trainX, m, '-q')
        p_labelsTest, p_accTest, p_valsTest = svm_predict(testY, testX, m, '-q')
        t.add_row([paramConfig, p_accTrain[0], p_accTest[0]])
        
print(t.get_string(sortby="Test Accuracy", reversesort=True))

Radial Basis Function
+--------------------------------+--------------------+--------------------+
|           Parameters           |   Train Accuracy   |   Test Accuracy    |
+--------------------------------+--------------------+--------------------+
|   -t 2 -c 78125 -g 2.56e-06    | 85.33333333333334  |        85.0        |
|     -t 2 -c 625 -g 0.0016      |        90.0        |        85.0        |
|     -t 2 -c 625 -g 0.00032     |        86.0        |        85.0        |
|    -t 2 -c 3125 -g 6.4e-05     |        86.0        |        85.0        |
|     -t 2 -c 3125 -g 0.0016     | 92.66666666666666  |        85.0        |
|    -t 2 -c 3125 -g 0.00032     | 89.33333333333333  |        85.0        |
|  -t 2 -c 1953125 -g 5.12e-07   | 88.66666666666667  |        85.0        |
|   -t 2 -c 15625 -g 1.28e-05    |        86.0        |        85.0        |
|   -t 2 -c 78125 -g 1.28e-05    |        88.0        | 84.16666666666667  |
|   -t 2 -c 78125 -g 1.024e-07   |        84.0        

In [14]:
print('Sigmoid')
t = PrettyTable(['Parameters', 'Train Accuracy', 'Test Accuracy'])

for cExp in range(-10,11):
    for gammaExp in range(-10,6):
        for coef0Exp in range(-10,6):
            paramConfig = '-t 2 -c '+str(5**cExp)+' -g '+str(5**gammaExp)+' -r '+str(5**coef0Exp)
            param = svm_parameter(paramConfig)
            m = svm_train(prob, param)
            p_labelsTrain, p_accTrain, p_valsTrain = svm_predict(trainY, trainX, m, '-q')
            p_labelsTest, p_accTest, p_valsTest = svm_predict(testY, testX, m, '-q')
            t.add_row([paramConfig, p_accTrain[0], p_accTest[0]])
            
print(t.get_string(sortby="Test Accuracy", reversesort=True))

Sigmoid
+---------------------------------------------+--------------------+--------------------+
|                  Parameters                 |   Train Accuracy   |   Test Accuracy    |
+---------------------------------------------+--------------------+--------------------+
|       -t 2 -c 78125 -g 2.56e-06 -r 625      | 85.33333333333334  |        85.0        |
|     -t 2 -c 78125 -g 2.56e-06 -r 6.4e-05    | 85.33333333333334  |        85.0        |
|    -t 2 -c 78125 -g 2.56e-06 -r 5.12e-07    | 85.33333333333334  |        85.0        |
|        -t 2 -c 78125 -g 2.56e-06 -r 5       | 85.33333333333334  |        85.0        |
|      -t 2 -c 78125 -g 2.56e-06 -r 3125      | 85.33333333333334  |        85.0        |
|       -t 2 -c 78125 -g 2.56e-06 -r 25       | 85.33333333333334  |        85.0        |
|    -t 2 -c 78125 -g 2.56e-06 -r 2.56e-06    | 85.33333333333334  |        85.0        |
|       -t 2 -c 78125 -g 2.56e-06 -r 125      | 85.33333333333334  |        85.0        |
| 

## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [15]:
print('Linear')
t = PrettyTable(['Parameters', '#Support Vector'])

for cExp in range(-10,11):
    paramConfig = '-t 0 -c '+str(5**cExp)
    param = svm_parameter(paramConfig)
    m = svm_train(prob, param)
    t.add_row([paramConfig, m.get_nr_sv()])
    
print(t)

Linear
+-------------------+-----------------+
|     Parameters    | #Support Vector |
+-------------------+-----------------+
| -t 0 -c 1.024e-07 |       140       |
|  -t 0 -c 5.12e-07 |       140       |
|  -t 0 -c 2.56e-06 |       140       |
|  -t 0 -c 1.28e-05 |       140       |
|  -t 0 -c 6.4e-05  |       140       |
|  -t 0 -c 0.00032  |       140       |
|   -t 0 -c 0.0016  |       140       |
|   -t 0 -c 0.008   |       124       |
|    -t 0 -c 0.04   |        87       |
|    -t 0 -c 0.2    |        68       |
|     -t 0 -c 1     |        58       |
|     -t 0 -c 5     |        54       |
|     -t 0 -c 25    |        50       |
|    -t 0 -c 125    |        50       |
|    -t 0 -c 625    |        49       |
|    -t 0 -c 3125   |        49       |
|   -t 0 -c 15625   |        49       |
|   -t 0 -c 78125   |        53       |
|   -t 0 -c 390625  |        67       |
|  -t 0 -c 1953125  |        83       |
|  -t 0 -c 9765625  |        64       |
+-------------------+------------

In [16]:
print('Polynomial')
t = PrettyTable(['Parameters', '#Support Vector'])

for cExp in range(-10,11):
    paramConfig = '-t 1 -c '+str(5**cExp)+' -g 0.00032 -r 0.2 -d 4'
    param = svm_parameter(paramConfig)
    m = svm_train(prob, param)
    t.add_row([paramConfig, m.get_nr_sv()])
    
print(t)

Polynomial
+------------------------------------------+-----------------+
|                Parameters                | #Support Vector |
+------------------------------------------+-----------------+
| -t 1 -c 1.024e-07 -g 0.00032 -r 0.2 -d 4 |       140       |
| -t 1 -c 5.12e-07 -g 0.00032 -r 0.2 -d 4  |       140       |
| -t 1 -c 2.56e-06 -g 0.00032 -r 0.2 -d 4  |       140       |
| -t 1 -c 1.28e-05 -g 0.00032 -r 0.2 -d 4  |       140       |
|  -t 1 -c 6.4e-05 -g 0.00032 -r 0.2 -d 4  |       140       |
|  -t 1 -c 0.00032 -g 0.00032 -r 0.2 -d 4  |       140       |
|  -t 1 -c 0.0016 -g 0.00032 -r 0.2 -d 4   |       140       |
|   -t 1 -c 0.008 -g 0.00032 -r 0.2 -d 4   |       140       |
|   -t 1 -c 0.04 -g 0.00032 -r 0.2 -d 4    |       140       |
|    -t 1 -c 0.2 -g 0.00032 -r 0.2 -d 4    |       140       |
|     -t 1 -c 1 -g 0.00032 -r 0.2 -d 4     |       140       |
|     -t 1 -c 5 -g 0.00032 -r 0.2 -d 4     |       140       |
|    -t 1 -c 25 -g 0.00032 -r 0.2 -d 4     |

In [17]:
print('Radial Basis Function')
t = PrettyTable(['Parameters', '#Support Vector'])

for cExp in range(-10,11):
    paramConfig = '-t 2 -c '+str(5**cExp)+' -g 2.56e-06'
    param = svm_parameter(paramConfig)
    m = svm_train(prob, param)
    t.add_row([paramConfig, m.get_nr_sv()])
    
print(t)

Radial Basis Function
+-------------------------------+-----------------+
|           Parameters          | #Support Vector |
+-------------------------------+-----------------+
| -t 2 -c 1.024e-07 -g 2.56e-06 |       140       |
|  -t 2 -c 5.12e-07 -g 2.56e-06 |       140       |
|  -t 2 -c 2.56e-06 -g 2.56e-06 |       140       |
|  -t 2 -c 1.28e-05 -g 2.56e-06 |       140       |
|  -t 2 -c 6.4e-05 -g 2.56e-06  |       140       |
|  -t 2 -c 0.00032 -g 2.56e-06  |       140       |
|   -t 2 -c 0.0016 -g 2.56e-06  |       140       |
|   -t 2 -c 0.008 -g 2.56e-06   |       140       |
|    -t 2 -c 0.04 -g 2.56e-06   |       140       |
|    -t 2 -c 0.2 -g 2.56e-06    |       140       |
|     -t 2 -c 1 -g 2.56e-06     |       140       |
|     -t 2 -c 5 -g 2.56e-06     |       140       |
|     -t 2 -c 25 -g 2.56e-06    |       140       |
|    -t 2 -c 125 -g 2.56e-06    |       140       |
|    -t 2 -c 625 -g 2.56e-06    |       140       |
|    -t 2 -c 3125 -g 2.56e-06   |       10

In [18]:
print('Sigmoid')
t = PrettyTable(['Parameters', '#Support Vector'])

for cExp in range(-10,30):
    paramConfig = '-t 3 -c '+str(5**cExp)+' -g 2.56e-06 -r 625'
    param = svm_parameter(paramConfig)
    m = svm_train(prob, param)
    t.add_row([paramConfig, m.get_nr_sv()])
    
print(t)

Sigmoid
+--------------------------------------------------+-----------------+
|                    Parameters                    | #Support Vector |
+--------------------------------------------------+-----------------+
|       -t 3 -c 1.024e-07 -g 2.56e-06 -r 625       |       140       |
|       -t 3 -c 5.12e-07 -g 2.56e-06 -r 625        |       140       |
|       -t 3 -c 2.56e-06 -g 2.56e-06 -r 625        |       140       |
|       -t 3 -c 1.28e-05 -g 2.56e-06 -r 625        |       140       |
|        -t 3 -c 6.4e-05 -g 2.56e-06 -r 625        |       140       |
|        -t 3 -c 0.00032 -g 2.56e-06 -r 625        |       140       |
|        -t 3 -c 0.0016 -g 2.56e-06 -r 625         |       140       |
|         -t 3 -c 0.008 -g 2.56e-06 -r 625         |       140       |
|         -t 3 -c 0.04 -g 2.56e-06 -r 625          |       140       |
|          -t 3 -c 0.2 -g 2.56e-06 -r 625          |       140       |
|           -t 3 -c 1 -g 2.56e-06 -r 625           |       140       

## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [24]:
import random
import numpy as np

WBeforeRemove = np.zeros(13)

for i in range(13):
    for j, index in enumerate(m.get_sv_indices()):
        WBeforeRemove[i] += m.get_sv_coef()[j][0]*m.get_SV()[j][i]
        
bBeforeRemove = m.rho[0]

svIndices = m.get_sv_indices()
notSVIndices = [i for i in range(len(trainX)) if i not in svIndices]

randomSVIndex = random.choice(svIndices)
randomNotSVIndex = random.choice(notSVIndices)

trainXSVRemove = [trainX[i] for i in range(len(trainX)) if i != randomSVIndex]
trainYSVRemove = [trainY[i] for i in range(len(trainY)) if i != randomSVIndex]

trainXNotSVRemove = [trainX[i] for i in range(len(trainX)) if i != randomNotSVIndex]
trainYNotSVRemove = [trainY[i] for i in range(len(trainY)) if i != randomNotSVIndex]

#SV Removed

prob = svm_problem(trainYSVRemove, trainXSVRemove,isKernel=True)
paramConfig = '-t 0 -c '+str(5**12)
param = svm_parameter(paramConfig)
mSVRemoved = svm_train(prob, param)

WAfterSVRemove = np.zeros(13)

for i in range(13):
    for j, index in enumerate(mSVRemoved.get_sv_indices()):
        WAfterSVRemove[i] += mSVRemoved.get_sv_coef()[j][0]*mSVRemoved.get_SV()[j][i]
        
bAfterSVRemove = mSVRemoved.rho[0]

#Not SV Removed
prob = svm_problem(trainYNotSVRemove, trainXNotSVRemove,isKernel=True)
paramConfig = '-t 0 -c '+str(5**12)
param = svm_parameter(paramConfig)
mNotSVRemoved = svm_train(prob, param)

WAfterNotSVRemove = np.zeros(13)

for i in range(13):
    for j, index in enumerate(mNotSVRemoved.get_sv_indices()):
        WAfterNotSVRemove[i] += mNotSVRemoved.get_sv_coef()[j][0]*mNotSVRemoved.get_SV()[j][i]
        
bAfterNotSVRemove = mNotSVRemoved.rho[0]

In [25]:
print(WBeforeRemove)
print()
print(WAfterSVRemove)
print()
print(WAfterNotSVRemove)

[ 11.81511334  -2.85874663   0.88258388  -7.61463158 -63.45462169
  -1.70441247  14.94278092 -45.66543324  -0.50450426   6.63253932
  18.6214625   18.06604945  22.01729072]

[ 23.62644058  -8.38495749  10.82723107 -13.73988259 -49.97236831
  -4.10436445   3.25056503  -4.03859975  -2.50569257 -46.18835178
  26.01612271  27.05269946 -13.82116646]

[  7.45042119 -16.56251835  65.53887194  -8.00365812 -79.13106184
  11.16057143  -0.23705383 -19.38467039   4.34327757 -36.44012244
  32.31453734  14.18840368  19.95757153]


### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 

In [27]:
from cvxopt import matrix, solvers
import numpy as np

d = len(trainX[0])

Q = np.zeros((d+1,d+1))
for i in range(1,d+1):
    for j in range(1,d+1):
        if i == j:
            Q[i][j] = 1
Q = matrix(Q)

p = np.zeros((d+1))
p = matrix(p)

A = np.zeros((len(trainY), len(trainX[0])+1))

for i in range(len(trainY)):
    A[i][0] = trainY[i]
    for j in range(len(trainX[i])):
        A[i][j+1] = trainX[i][j]
        
A = matrix(A)

c = np.ones((len(trainY)))
c = matrix(c)

sol = solvers.qp(Q,p,A,c)


print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  4.3692e-01 -1.1676e+01  4e+02  1e+00  4e+02
 1:  5.0190e-02 -3.7492e+01  4e+01  1e-01  4e+01
 2:  5.1439e-02 -1.4472e+00  2e+00  4e-03  1e+00
 3:  5.8792e-03 -1.0963e-01  1e-01  1e-04  3e-02
 4:  2.1350e-05 -4.2654e-03  4e-03  1e-06  3e-04
 5:  2.2696e-09 -4.3516e-05  4e-05  1e-08  3e-06
 6:  2.2696e-13 -4.3514e-07  4e-07  1e-10  3e-08
 7:  2.2696e-17 -4.3514e-09  4e-09  1e-12  3e-10
Optimal solution found.
[ 1.83e-02]
[-2.92e-10]
[-1.75e-09]
[-2.07e-09]
[ 1.44e-09]
[ 2.14e-09]
[ 3.29e-09]
[ 6.56e-11]
[-6.68e-10]
[ 9.00e-10]
[ 3.13e-09]
[ 1.97e-09]
[ 2.25e-09]
[ 7.13e-10]



In [28]:
paramConfig = '-t 0'
param = svm_parameter(paramConfig)
m = svm_train(prob, param)


svIndices = m.get_sv_indices()
notSVIndices = [i for i in range(len(trainX)) if i not in svIndices]

randomSVIndex = random.choice(svIndices)
randomNotSVIndex = random.choice(notSVIndices)

trainXSVRemove = [trainX[i] for i in range(len(trainX)) if i != randomSVIndex]
trainYSVRemove = [trainY[i] for i in range(len(trainY)) if i != randomSVIndex]

trainXNotSVRemove = [trainX[i] for i in range(len(trainX)) if i != randomNotSVIndex]
trainYNotSVRemove = [trainY[i] for i in range(len(trainY)) if i != randomNotSVIndex]



d = len(trainXNotSVRemove[0])

Q = np.zeros((d+1,d+1))
for i in range(1,d+1):
    for j in range(1,d+1):
        if i == j:
            Q[i][j] = 1
Q = matrix(Q)

p = np.zeros((d+1))
p = matrix(p)

A = np.zeros((len(trainYNotSVRemove), len(trainXNotSVRemove[0])+1))
for i in range(len(trainYNotSVRemove)):
    A[i][0] = trainYNotSVRemove[i]
    for j in range(len(trainXNotSVRemove[i])):
        A[i][j+1] = trainXNotSVRemove[i][j]
A = matrix(A)

c = np.ones((len(trainYNotSVRemove)))
c = matrix(c)

sol = solvers.qp(Q,p,A,c)


print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  4.3323e-01 -1.1728e+01  4e+02  1e+00  4e+02
 1:  4.9149e-02 -3.7503e+01  4e+01  1e-01  4e+01
 2:  5.0667e-02 -1.4436e+00  1e+00  4e-03  1e+00
 3:  5.8753e-03 -1.0993e-01  1e-01  1e-04  3e-02
 4:  2.2126e-05 -4.3340e-03  4e-03  1e-06  3e-04
 5:  2.3574e-09 -4.4254e-05  4e-05  1e-08  3e-06
 6:  2.3575e-13 -4.4251e-07  4e-07  1e-10  3e-08
 7:  2.3575e-17 -4.4251e-09  4e-09  1e-12  3e-10
Optimal solution found.
[ 2.00e-02]
[-2.96e-10]
[-1.77e-09]
[-2.09e-09]
[ 1.46e-09]
[ 2.17e-09]
[ 3.39e-09]
[ 9.40e-11]
[-6.94e-10]
[ 9.44e-10]
[ 3.17e-09]
[ 2.03e-09]
[ 2.26e-09]
[ 7.46e-10]



In [29]:
d = len(trainXSVRemove[0])

Q = np.zeros((d+1,d+1))
for i in range(1,d+1):
    for j in range(1,d+1):
        if i == j:
            Q[i][j] = 1
Q = matrix(Q)

p = np.zeros((d+1))
p = matrix(p)

A = np.zeros((len(trainYSVRemove), len(trainXSVRemove[0])+1))
for i in range(len(trainYSVRemove)):
    A[i][0] = trainYSVRemove[i]
    for j in range(len(trainXSVRemove[i])):
        A[i][j+1] = trainXSVRemove[i][j]
A = matrix(A)

c = np.ones((len(trainYSVRemove)))
c = matrix(c)

sol = solvers.qp(Q,p,A,c)


print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  4.2740e-01 -1.1596e+01  4e+02  1e+00  4e+02
 1:  4.5002e-02 -3.6930e+01  4e+01  1e-01  4e+01
 2:  4.6696e-02 -1.4218e+00  1e+00  4e-03  1e+00
 3:  5.5438e-03 -1.0751e-01  1e-01  1e-04  3e-02
 4:  1.9753e-05 -4.1153e-03  4e-03  1e-06  3e-04
 5:  2.0919e-09 -4.1923e-05  4e-05  1e-08  3e-06
 6:  2.0919e-13 -4.1921e-07  4e-07  1e-10  3e-08
 7:  2.0919e-17 -4.1921e-09  4e-09  1e-12  3e-10
Optimal solution found.
[ 1.78e-02]
[-2.77e-10]
[-1.69e-09]
[-1.99e-09]
[ 1.37e-09]
[ 2.05e-09]
[ 3.18e-09]
[ 6.52e-11]
[-6.38e-10]
[ 8.68e-10]
[ 2.99e-09]
[ 1.88e-09]
[ 2.16e-09]
[ 6.76e-10]

